In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import glob

In [29]:
# pretty_names_dir = {'B_caccae': '$B. caccae$',
#                 'B_cellulosilyticus_WH2': '$B. cellulosilyticus$',
#                 'B_ovatus': '$B. ovatus$',
#                 'B_thetaiotaomicron': '$B. thetaiotaomicron$',
#                 'B_uniformis': '$B. uniformis$',
#                 'B_vulgatus': '$B. vulgatus$',
#                 'C_aerofaciens': '$C. aerofaciens$',
#                 'C_scindens': '$C. scindens$',
#                 'C_spiroforme': '$C. spiroforme$',
#                 # 'D_longicatena': '$D. longicatena$',
#                 'P_distasonis': '$P. distasonis$',
#                 'R_obeum': '$R. obeum$'}

cols = [col for i, col in enumerate(cm.tab20.colors) if i % 2 == 0]
cols.pop(7)
cols.append(cm.tab20b.colors[4])
cols.append(cm.tab20b.colors[1])
cols.append(cm.tab20b.colors[14])
cols.append(cm.tab20b.colors[10])
cols.append(cm.tab20b.colors[17])
cols.append(cm.Dark2.colors[0])
cols.append(cm.Dark2.colors[3])
cols.append(cm.tab20.colors[1])
cols.append(cm.tab20.colors[3])
cols.append(cm.tab20.colors[5])
cols.append(cm.tab20.colors[7])
cols.append(cm.tab20.colors[9])
cols.append(cm.tab20.colors[11])
my_cmap = cm.colors.ListedColormap(cols)

pretty_names_dir = {'B_caccae': 'Bca',
                'B_cellulosilyticus_WH2': 'Bce',
                'B_ovatus': 'Bov',
                'B_thetaiotaomicron': 'Bth',
                'B_uniformis': 'Bun',
                'B_vulgatus': 'Bvu',
                'C_aerofaciens': 'Cae',
                'C_scindens': 'Csc',
                'C_spiroforme': 'Csp',
                # 'D_longicatena': 'Dlo',
                'P_distasonis': 'Pdi',
                'R_obeum': 'Rob'
                }

pretty_names_dir_ = {'B. caccae': 'Bca',
                'B. cellulosilyticus': 'Bce',
                'B. ovatus': 'Bov',
                'B. thetaiotaomicron': 'Bth',
                'B. uniformis': 'Bun',
                'B. vulgatus': 'Bvu',
                'C. aerofaciens': 'Cae',
                'C. scindens': 'Csc',
                'C. spiroforme': 'Csp',
                # 'D. longicatena': 'Dlo',
                'P. distasonis': 'Pdi',
                'R. obeum': 'Rob'
                }

dic = {"Bc":"B. caccae", 
       "BcWH2":"B. cellulosilyticus",
       "Bo":"B. ovatus",
       "Bth":"B. thetaiotaomicron",
       "Bu":"B. uniformis",
       "Bv":"B. vulgatus",
       "Ca":"C. aerofaciens",
       "Csc":"C. scindens",
       "Csp":"C. spiroforme",
       # "Dl":"D. longicatena",
       "Pd":"P. distasonis",
       "Ro":"R. obeum"}

dic = {"Bc":"B_caccae", 
       "BcWH2":"B_cellulosilyticus_WH2",
       "Bo":"B_ovatus",
       "Bth":"B_thetaiotaomicron",
       "Bu":"B_uniformis",
       "Bv":"B_vulgatus",
       "Ca":"C_aerofaciens",
       "Csc":"C_scindens",
       "Csp":"C_spiroforme",
       # "Dl":"D_longicatena",
       "Pd":"P_distasonis",
       "Ro":"R_obeum"}

In [53]:
lf0_reads = pd.read_table('/home/acs98/Documents/BCEM/Tesis-maestria/masters-thesis/ecological-modeling/mcnulty-datasets/LF0/reads.tsv', index_col=0)
hf0_reads = pd.read_table('/home/acs98/Documents/BCEM/Tesis-maestria/masters-thesis/ecological-modeling/mcnulty-datasets/HF0/reads.tsv', index_col=0)

In [78]:
lf0_eggNOGs = []
for file in dic.values():
    eggNOG = pd.read_table(f'eggNOG/{file}_eggNOG.tsv', header = 0, skiprows=4, skipfooter=3, engine='python')
    eggNOG['Species'] = eggNOG['#query'].str.split('_', expand=True)[0]
    eggNOG['Species'] = eggNOG['Species'].map(dic)
    eggNOG = eggNOG.loc[:,['Species', 'KEGG_ko']]
    eggNOG.set_index('Species', inplace=True)
    eggNOG = eggNOG['KEGG_ko'].str.split(',', expand=True)
    eggNOG = eggNOG.reset_index()
    eggNOG = eggNOG.melt(id_vars=['Species'], value_vars=list(range(eggNOG.shape[1]-1)), value_name='KEGG ortholog').dropna()
    eggNOG = eggNOG[eggNOG['KEGG ortholog'].isin(['', '-']) == False].loc[:,['Species', 'KEGG ortholog']].set_index('Species')
    eggNOG['KEGG ortholog'] = eggNOG['KEGG ortholog'].replace('ko:', '', regex=True)
    eggNOG.reset_index(inplace=True)
    for i in range(7):
        eggNOG[f'Sample{i+1}'] = lf0_reads[f'0{i+1}-09'][f'{file}']
    for i in range(7):
        eggNOG[f'Sample{i+1+7}'] = lf0_reads[f'0{i+1}-25'][f'{file}']
    # eggNOG.to_csv('test_gomixer.tsv', sep='\t', index=True)
    lf0_eggNOGs.append(eggNOG)
lf0_eggNOGs = pd.concat(lf0_eggNOGs, axis=0, ignore_index=True)
lf0_eggNOGs.index.name = 'id'
# lf0_eggNOGs.to_csv('~/Downloads/lf0_kos.tsv', sep='\t', index=True)
lf0_eggNOGs

,Species,KEGG ortholog,Sample1,Sample2,Sample3,Sample4,Sample5,Sample6,Sample7,Sample8,Sample9,Sample10,Sample11,Sample12,Sample13,Sample14
id,,,,,,,,,,,,,,,,
0,B_caccae,K09181,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,71570.5,31195.3,32701.1,67846.0,43071.5,96502.3
1,B_caccae,K21572,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,71570.5,31195.3,32701.1,67846.0,43071.5,96502.3
2,B_caccae,K21572,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,71570.5,31195.3,32701.1,67846.0,43071.5,96502.3
3,B_caccae,K09955,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,71570.5,31195.3,32701.1,67846.0,43071.5,96502.3
4,B_caccae,K07391,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,71570.5,31195.3,32701.1,67846.0,43071.5,96502.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18618,R_obeum,K07668,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,2258.2,730.5,436.9,4630.1,959.2,3424.1
18619,R_obeum,K13542,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,2258.2,730.5,436.9,4630.1,959.2,3424.1
18620,R_obeum,K13950,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,2258.2,730.5,436.9,4630.1,959.2,3424.1


In [79]:
hf0_eggNOGs = []
for file in dic.values():
    eggNOG = pd.read_table(f'eggNOG/{file}_eggNOG.tsv', header = 0, skiprows=4, skipfooter=3, engine='python')
    eggNOG['Species'] = eggNOG['#query'].str.split('_', expand=True)[0]
    eggNOG['Species'] = eggNOG['Species'].map(dic)
    eggNOG = eggNOG.loc[:,['Species', 'KEGG_ko']]
    eggNOG.set_index('Species', inplace=True)
    eggNOG = eggNOG['KEGG_ko'].str.split(',', expand=True)
    eggNOG = eggNOG.reset_index()
    eggNOG = eggNOG.melt(id_vars=['Species'], value_vars=list(range(eggNOG.shape[1]-1)), value_name='KEGG ortholog').dropna()
    eggNOG = eggNOG[eggNOG['KEGG ortholog'].isin(['', '-']) == False].loc[:,['Species', 'KEGG ortholog']].set_index('Species')
    eggNOG['KEGG ortholog'] = eggNOG['KEGG ortholog'].replace('ko:', '', regex=True)
    eggNOG.reset_index(inplace=True)
    for i in range(8):
        if i < 2:
            eggNOG[f'Sample{i+15}'] = hf0_reads[f'0{i+8}-09'][f'{file}']
        else:
            eggNOG[f'Sample{i+15}'] = hf0_reads[f'{i+8}-09'][f'{file}']
    for i in range(8):
        if i < 2:
            eggNOG[f'Sample{i+15+8}'] = hf0_reads[f'0{i+8}-25'][f'{file}']
        else:
            eggNOG[f'Sample{i+15+8}'] = hf0_reads[f'{i+8}-25'][f'{file}']
    hf0_eggNOGs.append(eggNOG)
hf0_eggNOGs = pd.concat(hf0_eggNOGs, axis=0, ignore_index=True)
hf0_eggNOGs.index.name = 'id'
# hf0_eggNOGs.to_csv('~/Downloads/hf0_kos.tsv', sep='\t', index=True)
hf0_eggNOGs

,Species,KEGG ortholog,Sample15,Sample16,Sample17,Sample18,Sample19,Sample20,Sample21,Sample22,Sample23,Sample24,Sample25,Sample26,Sample27,Sample28,Sample29,Sample30
id,,,,,,,,,,,,,,,,,,
0,B_caccae,K09181,83374.9,56049.2,47402.2,68332.2,108134.3,105486.6,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
1,B_caccae,K21572,83374.9,56049.2,47402.2,68332.2,108134.3,105486.6,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
2,B_caccae,K21572,83374.9,56049.2,47402.2,68332.2,108134.3,105486.6,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
3,B_caccae,K09955,83374.9,56049.2,47402.2,68332.2,108134.3,105486.6,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
4,B_caccae,K07391,83374.9,56049.2,47402.2,68332.2,108134.3,105486.6,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18618,R_obeum,K07668,1897.6,6666.4,3634.3,6908.4,7826.1,8073.8,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6
18619,R_obeum,K13542,1897.6,6666.4,3634.3,6908.4,7826.1,8073.8,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6
18620,R_obeum,K13950,1897.6,6666.4,3634.3,6908.4,7826.1,8073.8,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6


In [83]:
eggNOGs = pd.concat([lf0_eggNOGs, hf0_eggNOGs.iloc[:,2:]], axis=1)
eggNOGs.to_csv('gomixer_kos.tsv', sep='\t', index=True)
eggNOGs

,Species,KEGG ortholog,Sample1,Sample2,Sample3,Sample4,Sample5,Sample6,Sample7,Sample8,...,Sample21,Sample22,Sample23,Sample24,Sample25,Sample26,Sample27,Sample28,Sample29,Sample30
id,,,,,,,,,,,,,,,,,,,,,
0,B_caccae,K09181,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,...,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
1,B_caccae,K21572,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,...,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
2,B_caccae,K21572,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,...,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
3,B_caccae,K09955,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,...,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
4,B_caccae,K07391,8934.0,16347.9,3311.4,10565.4,22838.7,31839.2,15243.9,61652.2,...,76395.7,109233.5,21345.4,30936.9,27677.4,2503.2,4291.5,8294.3,3830.2,28049.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18618,R_obeum,K07668,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,...,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6
18619,R_obeum,K13542,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,...,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6
18620,R_obeum,K13950,372.5,7280.3,1000.6,12233.8,11485.2,8378.5,8433.5,1771.9,...,3918.0,9016.8,7484.9,11836.1,7021.0,1573.6,1097.0,7001.7,2093.3,10166.6
